# 测试一下损失函数中加入积分能不能行

In [ ]:
import jax
import jax.numpy as jnp
from jax import config, grad, jit, vmap
from pyswarm import pso
from jax.scipy.optimize import minimize
config.update("jax_enable_x64", True)

MAX_NPARAMS = 10
initial_params = jnp.ones(MAX_NPARAMS, dtype=jnp.float64)

def evaluate(data: dict, params=initial_params) -> dict:
    inputs, outputs = data['inputs'], data['outputs']
    n_dim = inputs.shape[1] // 2
    q, q_t = inputs[:, :n_dim], inputs[:, n_dim:]
    true_accelerations = outputs

    @jit
    def compute_acceleration(q, q_t, params):
        def lag(q, q_t, params):
            return equation(q, q_t, params)
        hessian_q_t = jax.hessian(lag, 1)(q, q_t, params)
        grad_q = jax.grad(lag, 0)(q, q_t, params)
        jacobian_q_q_t = jax.jacobian(jax.grad(lag, 1), 0)(q, q_t, params)
        return jnp.linalg.pinv(hessian_q_t) @ (grad_q - jacobian_q_q_t @ q_t)

    batch_compute_acceleration = jit(vmap(compute_acceleration, (0, 0, None)))

    @jit
    def loss_fn(params):
        pred = batch_compute_acceleration(q, q_t, params)
        return jnp.mean(jnp.square(pred - true_accelerations))

    def run_optimization(objective_fn, initial_guess):
        if initial_guess.size > MAX_NPARAMS:
            result = minimize(objective_fn, initial_guess,
                            method='BFGS', options={'maxiter': 500})
            return result.x
        else:
            def pso_wrapper(x):
                return objective_fn(jnp.array(x))
            
            lb = [-1.0]*initial_guess.size
            ub = [10.0]*initial_guess.size
            
            pso_params, _ = pso(pso_wrapper, lb, ub, 
                            swarmsize=30, maxiter=200)
            
            result = minimize(objective_fn, jnp.array(pso_params),
                            method='BFGS', options={'maxiter': 500})
            return result.x


    # 敏感度分析模块
    def calculate_sensitivities(opt_params, base_loss):
        mask = 1 - jnp.eye(MAX_NPARAMS)
        
        @jit
        def batch_loss(params_matrix):
            return vmap(loss_fn)(params_matrix * mask)

        def sensitivity_objective(flat_params):
            matrix_params = flat_params.reshape(MAX_NPARAMS, MAX_NPARAMS)
            return jnp.sum(batch_loss(matrix_params))

        try:
            initial_flat = (opt_params * mask).flatten()
            optimized_flat = run_optimization(sensitivity_objective, initial_flat)
            optimized_matrix = optimized_flat.reshape(MAX_NPARAMS, MAX_NPARAMS)
            losses = batch_loss(optimized_matrix)
            relative_loss = (losses - base_loss) / base_loss
            return {i: float(jnp.nan_to_num(relative_loss[i])) for i in range(MAX_NPARAMS)}
            
        except Exception as e:
            print(f"Sensitivity analysis error: {str(e)}")
            return {i: 0.0 for i in range(MAX_NPARAMS)}


    # 主流程
    try:
        optimized_params = run_optimization(loss_fn, params)
        final_loss = loss_fn(optimized_params)
    except Exception as e:
        print(f"Optimization failed: {e}")
        return None

    if not jnp.isfinite(final_loss):
        return None

    return {
        'params': optimized_params,
        'loss': final_loss,
        'sensitivities': calculate_sensitivities(optimized_params, final_loss)
    }

@jit
def equation(q: jnp.array, q_t: jnp.array, params: jnp.array) -> jnp.array:
    q1, q2 = q[...,0], q[...,1]
    q1_t, q2_t = q_t[...,0], q_t[...,1]
    
    T = params[0]*(q1_t**2 + q2_t**2) 
    V = - params[1]/jnp.sqrt(q1**2 + q2**2)

    return T - V


In [ ]:
from jax import random

import pandas as pd
# 读取 CSV 文件并转换为 NumPy 数组
data0 = pd.read_csv('./xy_orbit.csv')#
tae = data0.to_numpy()

# 使用 JAX 的数组操作替换 PyTorch 的操作
t = jnp.array(tae[:, 0], dtype=jnp.float64)  # 转换为 JAX 数组
state = jnp.array(tae[:, 1:], dtype=jnp.float64)  # 转换为 JAX 数组
print(state)
print(t)
# 将数据存储在字典中
data = {
    'times': t,
    'states': state
}



print(initial_params)
# 评估并优化参数
final_loss = evaluate(data, initial_params)
print("最终损失值 (MSE):", final_loss)

In [ ]:
import jax
import jax.numpy as jnp
from jax import config, grad, jit, vmap
from jax.experimental.ode import odeint
from pyswarm import pso
from jax.scipy.optimize import minimize
import traceback
config.update("jax_enable_x64", True)

# --- Global Constants ---
# Define MAX_NPARAMS, used in run_optimization logic
# Adjust this value based on your system's typical parameter count
# or the threshold where you want to switch optimization methods.
MAX_NPARAMS = 10
initial_params = jnp.ones(MAX_NPARAMS, dtype=jnp.float64)

def evaluate(data: dict, params: jnp.array) -> dict:
    times = data['times']
    target_states = data['states']
    initial_state = target_states[0]
    n_dim = target_states.shape[1] // 2

    def compute_acceleration(q, q_t, params_acc):
        def lag(q_lag, q_t_lag, params_lag):
            return equation(q_lag, q_t_lag, params_lag)
        try:
            hessian_q_t = jax.hessian(lag, 1)(q, q_t, params_acc)
            grad_q = jax.grad(lag, 0)(q, q_t, params_acc)
            jacobian_q_q_t = jax.jacobian(jax.grad(lag, 1), 0)(q, q_t, params_acc)
            inv_hessian = jnp.linalg.pinv(hessian_q_t, rtol=1e-10)
            accel = inv_hessian @ (grad_q - jacobian_q_q_t @ q_t)
        except jnp.linalg.LinAlgError:
            print(f"Warning: Singular matrix in acceleration computation.")
            accel = jnp.full_like(q, jnp.nan)
        return accel

    @jit
    def dynamics_func(y, t, params_dyn):
        q = y[:n_dim]
        q_t = y[n_dim:]
        q_tt = compute_acceleration(q, q_t, params_dyn)
        return jnp.concatenate([q_t, q_tt])

    @jit
    def loss_fn(params_loss):
        pred_states = odeint(dynamics_func, initial_state, times, params_loss, rtol=1e-6, atol=1e-6)
        loss = jnp.mean(jnp.square(pred_states - target_states))
        return loss

    def run_optimization(objective_fn, initial_guess):
        n_params_opt = initial_guess.size
        print(f"Running optimization for {n_params_opt} parameters.")

        if n_params_opt > MAX_NPARAMS:
            print("Using BFGS optimizer.")
            try:
                result = minimize(objective_fn, initial_guess, method='BFGS', options={'maxiter': 500})
                if not result.success: 
                    print(f"Warning: BFGS optimization did not converge. Status: {result.status}")
                return result.x
            except Exception as e:
                print(f"BFGS optimization failed: {e}")
                # 返回初始猜测值，避免完全失败
                return initial_guess
        else:
            print("Using PSO + BFGS optimizer.")
            def pso_wrapper(x):
                return float(objective_fn(jnp.array(x, dtype=initial_guess.dtype)))

            # 使用合理的搜索范围
            lb = jnp.full(n_params_opt, -1.0, dtype=initial_guess.dtype).tolist()
            ub = jnp.full(n_params_opt, 10.0, dtype=initial_guess.dtype).tolist()

            try:
                print("Starting PSO optimization...")
                pso_params, pso_loss = pso(pso_wrapper, lb, ub, swarmsize=50, maxiter=200, debug=True)
                pso_params = jnp.array(pso_params, dtype=initial_guess.dtype)
                print(f"PSO finished with loss: {pso_loss}. Refining with BFGS...")
                
                try:
                    result = minimize(objective_fn, pso_params, method='BFGS', options={'maxiter': 500})
                    if not result.success:
                        print(f"Warning: BFGS (post-PSO) did not converge. Status: {result.status}")
                    print(f"BFGS refinement complete. Final loss: {objective_fn(result.x)}")
                    return result.x
                except Exception as e:
                    print(f"BFGS refinement failed: {e}, returning PSO results.")
                    return pso_params
                
            except Exception as e:
                print(f"PSO optimization failed: {e}. Falling back to BFGS.")
                try:
                    result = minimize(objective_fn, initial_guess, method='BFGS', options={'maxiter': 500})
                    if not result.success:
                        print(f"Warning: Fallback BFGS optimization did not converge. Status: {result.status}")
                    return result.x
                except Exception as e:
                    print(f"Fallback BFGS also failed: {e}")
                    return initial_guess  # 返回初始猜测值，避免完全失败

    # --- Sensitivity Analysis (Corrected Logic) ---
    def calculate_sensitivities(opt_params, base_loss):
        n_params_sens = opt_params.size
        if n_params_sens == 0:
            print("Warning: Skipping sensitivity analysis. No parameters found.")
            return {}

        print(f"Calculating sensitivities for {n_params_sens} parameters (re-optimization method)...")

        # Use vmap for batch loss calculation
        @jit
        def batch_loss(params_matrix):
            # Define the single evaluation loss function again, taking params only
            def single_loss_eval(p):
                pred_states = odeint(dynamics_func, initial_state, times, p, rtol=1e-6, atol=1e-6)
                pred_states = jnp.nan_to_num(pred_states, nan=jnp.inf)
                return jnp.mean(jnp.square(pred_states - target_states))
            # Ensure vmap operates correctly on the parameters dimension
            return vmap(single_loss_eval)(params_matrix)

        try:
            # Create mask based on the actual number of optimized parameters
            mask = 1.0 - jnp.eye(n_params_sens, dtype=opt_params.dtype)
            # Create matrix where row 'i' has param 'i' = 0
            # Broadcast opt_params (shape [n]) with mask (shape [n, n])
            masked_params = jnp.expand_dims(opt_params, 0) * mask

            # Calculate loss for each zeroed case
            losses_if_zero = batch_loss(masked_params)

            # Calculate relative loss change, adding epsilon for numerical stability
            relative_loss_change = (losses_if_zero - base_loss) / (base_loss + 1e-15)

            # Build sensitivity dictionary using the actual number of parameters
            sensitivities = {i: float(jnp.nan_to_num(relative_loss_change[i])) for i in range(n_params_sens)}

            print("Sensitivities calculated based on zeroing each parameter.")
            return sensitivities

        except Exception as e:
            print(f"Sensitivity analysis error: {str(e)}")
            print(traceback.format_exc())
            # Return zero sensitivities for all actual parameters on error
            return {i: 0.0 for i in range(n_params_sens)}

    # --- Main Execution Flow ---
    print("Starting parameter evaluation...")
    try:
        # Ensure initial params have a float dtype for optimization
        initial_params_float = jnp.asarray(params, dtype=jnp.float64)
        optimized_params = run_optimization(loss_fn, initial_params_float)
        print(f"Optimization finished. Optimal parameters: {optimized_params}")
        # Ensure optimized params are used with correct dtype for final loss calculation
        final_loss = loss_fn(jnp.asarray(optimized_params, dtype=jnp.float64))
        print(f"Final loss: {final_loss}")

    except Exception as e:
        print(f"Optimization failed: {e}")
        print(traceback.format_exc())
        return None # Indicate failure

    # Check if optimization produced valid results before sensitivity analysis
    if optimized_params is None or not jnp.isfinite(final_loss) or final_loss == jnp.inf:
        print(f"Optimization resulted in invalid parameters or non-finite loss: {final_loss}. Skipping sensitivity analysis.")
        # Return results without sensitivities if optimization failed or loss is invalid
        return {
            'params': optimized_params, # Could be None
            'loss': float(final_loss) if jnp.isfinite(final_loss) else float('inf'),
            'sensitivities': {} # Return empty sensitivities
        }

    # Proceed with sensitivity analysis only if optimization was successful
    sensitivities = calculate_sensitivities(optimized_params, final_loss)

    return {
        'params': optimized_params,
        'loss': float(final_loss), # Convert scalar JAX array to float
        'sensitivities': sensitivities
    }

# Dummy equation function if not provided elsewhere
@jit
def equation(q: jnp.array, q_t: jnp.array, params: jnp.array) -> jnp.array:

    q1, q2 = q[..., 0], q[..., 1]
    q1_t, q2_t = q_t[..., 0], q_t[..., 1]

    T = 0.5 * params[0] * (q1_t**2 + q2_t**2) # Corrected Kinetic Energy form (factor of 0.5)
    V = - params[1] / jnp.sqrt(q1**2 + q2**2 + 1e-8) # Potential Energy (add epsilon for stability near origin)

    return T - V



In [ ]:
from jax import random
import jax.numpy as jnp
import pandas as pd

data0 = pd.read_csv('./xy_orbit.csv') # 
tae = data0.to_numpy()

# 使用 JAX 的数组操作
t = jnp.array(tae[:, 0], dtype=jnp.float64)/100
state = jnp.array(tae[:, 1:], dtype=jnp.float64)/1e10 # 

data = {
    'times': t,
    'states': state # Use the scaled states
}

initial_params = jnp.array([1.0] * 10, dtype=jnp.float64) 

print("Initial Parameters (for scaled system):", initial_params)

results = evaluate(data, initial_params) # Pass scaled data and initial guess

# --- Output Results ---
print("\n--- Evaluation Results ---")
if results:
    print(f"Optimized Parameters (for scaled system): {results['params']}")
    print(f"Final Loss (MSE, scaled units): {results['loss']}")
    print(f"Sensitivities (scaled units): {results['sensitivities']}")

# 问题很大，我们首先来验证一下计算是否可行。
我们首先验证计算的加速度是否正确。

In [ ]:
# 设置一个数据初始数据
from jax import random
import jax
import jax.numpy as jnp
import pandas as pd

#初始状态
initial = jnp.array([1.0, 2.0 , 3.0, 4.0])

#对应加速度
def dynamic(state):
    x = state[0]
    y = state[1]
    x_tt = -x/(x**2 + y**2)**(3/2)
    y_tt = -y/(x**2 + y**2)**(3/2)
    return jnp.array([state[2],state[3],x_tt, y_tt])

acceleration = dynamic(initial)

print("Initial State:", initial)
print("Acceleration:", acceleration)

#对应的方程
def equation(q: jnp.array, q_t: jnp.array, params: jnp.array) -> jnp.array:

    q1, q2 = q[..., 0], q[..., 1]
    q1_t, q2_t = q_t[..., 0], q_t[..., 1]

    T = params[0] *(q1_t**2 + q2_t**2) # Corrected Kinetic Energy form (factor of 0.5)
    V = - params[1] / jnp.sqrt(q1**2 + q2**2) # Potential Energy (add epsilon for stability near origin)

    return T - V

def compute_acceleration(q, q_t, params):
    def lag(q, q_t, params):
        return equation(q, q_t, params)
    hessian_q_t = jax.hessian(lag, 1)(q, q_t, params)
    grad_q = jax.grad(lag, 0)(q, q_t, params)
    jacobian_q_q_t = jax.jacobian(jax.grad(lag, 1), 0)(q, q_t, params)
    q_tt = jnp.linalg.pinv(hessian_q_t) @ (grad_q - jacobian_q_q_t @ q_t)
    return jnp.concatenate([q_t, q_tt])

params = jnp.array([0.5, 1.0])

print('acc',compute_acceleration(initial[:2],initial[2:], params))



Initial State: [1. 2. 3. 4.]
Acceleration: [ 3.          4.         -0.08944272 -0.17888544]
acc [-0.08944272 -0.17888544]


# 检验完毕，能够正确计算加速度。
接着我们开始积分，看看其能否使用自动加速度计算的式子积分

In [ ]:
import jax.numpy as jnp
from jax.experimental.ode import odeint
import matplotlib.pyplot as plt

t = jnp.linspace(0, 10, 100)  # 时间点，从 0 到 10，共 100 个点
y0 = initial

# 使用 odeint 求解
y1 = odeint(dynamic, y0, t)
y2 = odeint(compute_acceleration, y0, t, params)

# 可视化
plt.figure(figsize=(10, 6))
plt.plot(t, y1[:, 0], label='x1(t) (Oscillator 1)', color='blue')
plt.plot(t, y2[:, 1], label='x2(t) (Oscillator 2)', color='orange')
plt.xlabel('Time (t)')
plt.ylabel('Displacement')
plt.title('Coupled Harmonic Oscillators')
plt.legend()
plt.grid(True)
plt.show()
